In [8]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from IPython.display import display, clear_output


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import anndata as ad

import pandas as pd
import numpy as np
from collections import Counter


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report


from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb


from imblearn.over_sampling import SMOTE

In [9]:
rxrx19a_metadata = pd.read_csv('data/rxrx19a_metadata.csv', index_col=0)
rxrx19b_metadata = pd.read_csv('data/rxrx19b_metadata.csv', index_col=0)
rxrx19a_embeddings = pd.read_csv('data/rxrx19a_embeddings.csv', index_col=0)
rxrx19b_embeddings = pd.read_csv('data/rxrx19b_embeddings.csv', index_col=0)


rxrx19a_metadata.head()
rxrx19b_metadata.head()



,well_id,cell_type,experiment,plate,well,site,disease_condition,treatment,treatment_conc,SMILES
site_id,,,,,,,,,,
HRCE-1_1_AA02_1,HRCE-1_1_AA02,HRCE,HRCE-1,1,AA02,1,Active SARS-CoV-2,Flubendazole,0.1,COC(=O)NC1=NC2=C(N1)C=C(C=C2)C(=O)C1=CC=C(F)C=...
HRCE-1_1_AA02_2,HRCE-1_1_AA02,HRCE,HRCE-1,1,AA02,2,Active SARS-CoV-2,Flubendazole,0.1,COC(=O)NC1=NC2=C(N1)C=C(C=C2)C(=O)C1=CC=C(F)C=...
HRCE-1_1_AA02_3,HRCE-1_1_AA02,HRCE,HRCE-1,1,AA02,3,Active SARS-CoV-2,Flubendazole,0.1,COC(=O)NC1=NC2=C(N1)C=C(C=C2)C(=O)C1=CC=C(F)C=...
HRCE-1_1_AA02_4,HRCE-1_1_AA02,HRCE,HRCE-1,1,AA02,4,Active SARS-CoV-2,Flubendazole,0.1,COC(=O)NC1=NC2=C(N1)C=C(C=C2)C(=O)C1=CC=C(F)C=...
HRCE-1_1_AA03_1,HRCE-1_1_AA03,HRCE,HRCE-1,1,AA03,1,Active SARS-CoV-2,acetylcysteine,1.0,"CC(=O)N[C@H](CS)C(O)=O |a:4,r|"


,well_id,cell_type,experiment,plate,well,site,disease_condition,treatment,treatment_conc,SMILES
site_id,,,,,,,,,,
HUVEC-1_1_AA02_1,HUVEC-1_1_AA02,HUVEC,HUVEC-1,1,AA02,1,storm-severe,Ravoxertinib,1.00,CN1N=CC=C1NC1=NC=CC(=N1)C1=CC(=O)N(C=C1)C(CO)C...
HUVEC-1_1_AA03_1,HUVEC-1_1_AA03,HUVEC,HUVEC-1,1,AA03,1,storm-severe,Diphenylpyraline,0.25,"CN1CCC(CC1)OC(C1=CC=CC=C1)C1=CC=CC=C1 |c:12,14..."
HUVEC-1_1_AA04_1,HUVEC-1_1_AA04,HUVEC,HUVEC-1,1,AA04,1,storm-severe,Indirubin,1.00,O=C1NC2=CC=CC=C2\C1=C1\NC2=C(C=CC=C2)C1=O |c:5...
HUVEC-1_1_AA05_1,HUVEC-1_1_AA05,HUVEC,HUVEC-1,1,AA05,1,storm-severe,YO-01027,1.00,C[C@H](NC(=O)CC1=CC(F)=CC(F)=C1)C(=O)N[C@H]1C2...
HUVEC-1_1_AA07_1,HUVEC-1_1_AA07,HUVEC,HUVEC-1,1,AA07,1,storm-severe,taurine,1.00,NCCS(O)(=O)=O


In [10]:
X = rxrx19b_embeddings.values.tolist()

X_series = pd.Series(X, index=rxrx19b_embeddings.index, name='embeddings')
X_df = X_series.to_frame()

y = rxrx19b_metadata['treatment']
y_df = y.to_frame()
X_df = X_df.loc[y.index]

In [11]:
X_df.head()

y_df.head()

,embeddings
site_id,
HUVEC-1_1_AA02_1,"[-0.2086878, 4.063373, -0.12942116, -1.1610049..."
HUVEC-1_1_AA03_1,"[-0.05914755, 4.103718, -1.1349981, -0.7888756..."
HUVEC-1_1_AA04_1,"[-0.15590928, 3.754905, -0.36183846, -0.206010..."
HUVEC-1_1_AA05_1,"[-0.7296535, 3.4197803, 0.059896838, -0.312398..."
HUVEC-1_1_AA07_1,"[0.08298465, 4.188676, -0.16406447, -0.0579336..."


,treatment
site_id,
HUVEC-1_1_AA02_1,Ravoxertinib
HUVEC-1_1_AA03_1,Diphenylpyraline
HUVEC-1_1_AA04_1,Indirubin
HUVEC-1_1_AA05_1,YO-01027
HUVEC-1_1_AA07_1,taurine


In [12]:
non_nan_indices = ~y_df['treatment'].isna()
X_filtered = X_df[non_nan_indices]
y_filtered = y_df[non_nan_indices]

In [13]:
X = np.vstack(X_filtered['embeddings'].values)
y = y_filtered['treatment'].values

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

class_names = label_encoder.classes_

num_classes = len(np.unique(y_encoded))
print(f"Number of classes: {num_classes}")
print("Classes:", class_names)

Number of classes: 1856
Classes: ['(+)-Cedrol' '(+)-Ephedrine (hydrochloride)' '(+)-cloprostenol' ...
 'zofenopril-calcium' 'zolimidine' 'zuclopenthixol']


In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
print("Class weights:", class_weights_dict)

Class weights: {0: np.float64(0.9599249405469679), 1: np.float64(0.994207974137931), 2: np.float64(0.9599249405469679), 3: np.float64(0.9599249405469679), 4: np.float64(0.9599249405469679), 5: np.float64(0.9599249405469679), 6: np.float64(1.4651485934664248), 7: np.float64(0.9599249405469679), 8: np.float64(0.9599249405469679), 9: np.float64(0.994207974137931), 10: np.float64(0.9599249405469679), 11: np.float64(1.4651485934664248), 12: np.float64(0.9599249405469679), 13: np.float64(0.9599249405469679), 14: np.float64(0.9599249405469679), 15: np.float64(0.994207974137931), 16: np.float64(0.994207974137931), 17: np.float64(0.9599249405469679), 18: np.float64(0.9599249405469679), 19: np.float64(0.9599249405469679), 20: np.float64(0.9599249405469679), 21: np.float64(0.9599249405469679), 22: np.float64(0.994207974137931), 23: np.float64(0.994207974137931), 24: np.float64(0.9599249405469679), 25: np.float64(0.9599249405469679), 26: np.float64(0.9599249405469679), 27: np.float64(0.95992494054

In [15]:
sm = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = sm.fit_resample(X_train_scaled, y_train)
print("Resampled training set shape:", Counter(y_train_resampled))

Resampled training set shape: Counter({np.int64(1253): 57, np.int64(658): 57, np.int64(1855): 57, np.int64(191): 57, np.int64(466): 57, np.int64(584): 57, np.int64(1484): 57, np.int64(1619): 57, np.int64(1419): 57, np.int64(1792): 57, np.int64(214): 57, np.int64(1381): 57, np.int64(1306): 57, np.int64(313): 57, np.int64(1397): 57, np.int64(422): 57, np.int64(1153): 57, np.int64(1778): 57, np.int64(1436): 57, np.int64(926): 57, np.int64(986): 57, np.int64(1787): 57, np.int64(484): 57, np.int64(680): 57, np.int64(331): 57, np.int64(1213): 57, np.int64(335): 57, np.int64(305): 57, np.int64(555): 57, np.int64(742): 57, np.int64(883): 57, np.int64(1093): 57, np.int64(182): 57, np.int64(1043): 57, np.int64(417): 57, np.int64(558): 57, np.int64(616): 57, np.int64(477): 57, np.int64(97): 57, np.int64(157): 57, np.int64(1498): 57, np.int64(35): 57, np.int64(98): 57, np.int64(988): 57, np.int64(710): 57, np.int64(343): 57, np.int64(1814): 57, np.int64(1672): 57, np.int64(1678): 57, np.int64(993)

In [16]:
lr = LogisticRegression(
    class_weight='balanced', max_iter=1000, random_state=42
)
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)
print("\nLogistic Regression with Class Weights:")
print(classification_report(y_test, y_pred_lr, target_names=class_names))

# With SMOTE
lr_smote = LogisticRegression(max_iter=1000, random_state=42)
lr_smote.fit(X_train_resampled, y_train_resampled)
y_pred_lr_smote = lr_smote.predict(X_test_scaled)
print("Logistic Regression with SMOTE:")
print(classification_report(y_test, y_pred_lr_smote, target_names=class_names))

LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)


Logistic Regression with Class Weights:
                                                             precision    recall  f1-score   support

                                                 (+)-Cedrol       0.00      0.00      0.00         7
                              (+)-Ephedrine (hydrochloride)       0.00      0.00      0.00         8
                                           (+)-cloprostenol       0.00      0.00      0.00         7
                                     (-)-Huperzine A (HupA)       0.00      0.00      0.00         7
                                                (R)-Tegafur       0.00      0.00      0.00         7
                                               (R)-baclofen       0.00      0.00      0.00         7
                                 (S)-10-Hydroxycamptothecin       0.33      0.20      0.25         5
                                             (S)-crizotinib       1.00      0.29      0.44         7
1-(3,4-diethoxybenzyl)-6,7-diethoxy-3,4-dihydrois

/opt/miniconda/envs/proj/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/miniconda/envs/proj/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/miniconda/envs/proj/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


KeyboardInterrupt: 

In [ ]:
svm = SVC(
    class_weight='balanced', probability=True, random_state=42
)
svm.fit(X_train_scaled, y_train)
y_pred_svm = svm.predict(X_test_scaled)
print("\nSVM with Class Weights:")
print(classification_report(y_test, y_pred_svm, target_names=class_names))

# With SMOTE
svm_smote = SVC(probability=True, random_state=42)
svm_smote.fit(X_train_resampled, y_train_resampled)
y_pred_svm_smote = svm_smote.predict(X_test_scaled)
print("SVM with SMOTE:")
print(classification_report(y_test, y_pred_svm_smote, target_names=class_names))